# Twitter Named Entity Recognition Case Study

### About
Twitter is a microblogging and social networking service on which users post and interact with messages known as "tweets". Every second, on average, around 6,000 tweets are tweeted on Twitter, corresponding to over 350,000 tweets sent per minute, 500 million tweets per day.

### Problem statement 
Twitter wants to automatically tag and analyze tweets for better understanding of the trends and topics without being dependent on the hashtags that the users use. Many users do not use hashtags or sometimes use wrong or mis-spelled tags, so they want to completely remove this problem and create a system of recognizing important content of the tweets.

### Objective
Named Entity Recognition (NER) is an important subtask of information extraction that seeks to locate and recognise named entities.
We need to train models that will be able to identify the various named entities.

### Data
Dataset is annotated with 10 fine-grained NER categories: person, geo-location, company, facility, product,music artist, movie, sports team, tv show and other. Dataset was extracted from tweets and is structured in CoNLL format., in English language. Containing in Text file format.
The CoNLL format is a text file with one word per line with sentences separated by an empty line. The first word in a line should be the word and the last word should be the label.

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [23]:
import os
root_path = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
data_path = os.path.join(root_path,'data')
train_data_path = os.path.join(data_path,'wnut 16.txt.conll')
test_data_path = os.path.join(data_path,'wnut 16test.txt.conll')

## Getting the data

In [50]:
# reading the training file
with open(train_data_path,'r') as f:
    train_raw = f.read()
with open(test_data_path,'r') as f:
    test_raw = f.read()

In [51]:
# creating a function to format the data
def extract_ner_from_conll(conll_data):
    # Split the data into sentences based on empty lines
    sentences = [sentence.strip() for sentence in conll_data.strip().split('\n\n')]
    ner_data = []

    for sentence in sentences:
        tokenised_sentence = []
        for token_entity in sentence.split('\n'):
            token, entity = token_entity.split('\t')
            tokenised_sentence.append((token,entity))
        ner_data.append(tokenised_sentence)

    return ner_data

In [52]:
# preprocessing the raw files
train_data = extract_ner_from_conll(train_raw)
test_data = extract_ner_from_conll(test_raw)

In [68]:
# checking sentences after preprocessing
print(train_data[0])

[('@SammieLynnsMom', 'O'), ('@tg10781', 'O'), ('they', 'O'), ('will', 'O'), ('be', 'O'), ('all', 'O'), ('done', 'O'), ('by', 'O'), ('Sunday', 'O'), ('trust', 'O'), ('me', 'O'), ('*wink*', 'O')]


## EDA

In [98]:
# number of words in the vocabulary and lenght of sentences in the training data

sentence_lenghts = list()
word_set = set()
for sentence in train_data:
    sentence_lenghts.append(len(sentence))
    for word in sentence:
        word_set.add(word[0])
        
NUM_WORDS = len(word_set)+2 # +2 to include padding and out of vocabulary
print(f"Number of unique words in training data (including padding and OOV token) = {NUM_WORDS}")
print(f"Maximum sentence length = {max(sentence_lenghts)}")
print(f"Minimum sentence length = {min(sentence_lenghts)}")

Number of unique words in training data (including padding and OOV token) = 10588
Maximum sentence length = 39
Minimum sentence length = 1


In [126]:
# since the max sentence length if 39, we will take a length of 45 in our model to incorporate for edge cases in inference
SENTENCE_LENGTH = 45
# we will keep the embedding dimensions to be 50 since the number of datapoints is small
EMBEDDING_DIMS = 50

In [99]:
# number of entities
entity_set = set()
for sentence in train_data:
    for word in sentence:
        entity_set.add(word[1])
        
NUM_ENTITIES = len(entity_set)
print(f"Number of unique entities in training data = {NUM_ENTITIES}")

Number of unique entities in training data = 21


## Data preparation

In [182]:
# create a function to prepare the data to be fed into the model
# --> punctuation should be handled along with their entities
def prepare_data(text_data):
    
    # initialize empty lists for sentences and entities
    sentences = []
    entities = []
    
    for sentence in text_data:
        
        # initialize empty lists for sentence text and corresponding entities
        word_list = []
        entity_list = []
        
        for token in sentence:
            word_list.append(token[0])
            entity_list.append(token[1])
        
        sentences.append(word_list)
        entities.append(entity_list)
    
    # create a single string for each sentence and entity by joining elements with whitespace
    sentences = np.array([' '.join(sentence) for sentence in sentences])
    entities = np.array([' '.join(entity) for entity in entities])
    
    return (sentences,entities)

In [183]:
# call the prepare_data function on the train and test data
xtrain, ytrain = prepare_data(train_data)
xtest, ytest = prepare_data(test_data)

In [184]:
# checking sentences are conversion
print(xtrain[0])
print(ytrain[0])

@SammieLynnsMom @tg10781 they will be all done by Sunday trust me *wink*
O O O O O O O O O O O O


## Bidirectional LSTM + CRF

In [185]:
from tensorflow.keras.layers import Input, TextVectorization, Embedding, Bidirectional, LSTM, TimeDistributed, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow_addons.layers import CRF
from tensorflow_addons.losses import SigmoidFocalCrossEntropy

In [186]:
# adapt the vectorizer layer before modeling
sentence_tokenizer = TextVectorization(max_tokens=NUM_WORDS, output_sequence_length=SENTENCE_LENGTH, standardize='lower', name='sentence_tokenizer')
sentence_tokenizer.adapt(xtrain)

In [187]:
# define a function to build and compile model
def build_lstm_model(name='bi-lstm+crf'):
    
    # input layer for getting sentences
    sentence_input = Input(shape=(1,), dtype=tf.string, name='sentence_input')
    
    # tokenizing the sentence
    x = sentence_tokenizer(sentence_input)
    
    # creating embeddings for each token in sentence
    embeddings = Embedding(
        input_dim = NUM_WORDS,
        output_dim = EMBEDDING_DIMS,
        mask_zero = True,
        name = 'word_embedding'
    )(x)
    
    # stacking two bidirectional LSTMs
    output_sequence = Bidirectional(LSTM(40, return_sequences=True), name='lstm_1')(embeddings)
    output_sequence = Bidirectional(LSTM(40, return_sequences=True), name='lstm_2')(output_sequence)
    
    # passing the sequence through dense layer to compress the information
    dense_sequence = TimeDistributed(Dense(25, activation='relu'), name='dense')(output_sequence)
    
    # passing the dense sequences through crf layer
    predicted_sequence, potentials, sequence_length, crf_kernel = CRF(NUM_ENTITIES, name='crf')(dense_sequence)
    
    # define the train model
    training_model = Model(sentence_input, potentials)
    
    # compile the model
    training_model.compile(
        loss=SigmoidFocalCrossEntropy(),
        optimizer='adam'
    )
    
    # create an inference model
    inference_model = Model(sentence_input, predicted_sequence)
    
    return training_model, inference_model

In [188]:
# creating the traing and inferencing model
lstm_training_model, lstm_inference_model = build_lstm_model()

In [189]:
lstm_training_model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sentence_input (InputLayer  [(None, 1)]               0         
 )                                                               
                                                                 
 sentence_tokenizer (TextVe  (None, 45)                0         
 ctorization)                                                    
                                                                 
 word_embedding (Embedding)  (None, 45, 50)            529400    
                                                                 
 lstm_1 (Bidirectional)      (None, 45, 80)            29120     
                                                                 
 lstm_2 (Bidirectional)      (None, 45, 80)            38720     
                                                                 
 dense (TimeDistributed)     (None, 45, 25)            202

In [190]:
# prepare the entity data

# tokenizing the entities
entity_tokenizer = TextVectorization(max_tokens=NUM_ENTITIES, output_sequence_length=SENTENCE_LENGTH, standardize='lower', name='entity_tokenizer')
entity_tokenizer.adapt(ytrain)
ytrain_tokenized = entity_tokenizer(ytrain)

# one hot encoding the entitiy tokens
entity_ohe = Lambda(lambda x: tf.one_hot(x, NUM_ENTITIES))
ytrain_tokenized = entity_ohe(ytrain_tokenized)

In [156]:
ytrain_tokenized[0]

<tf.Tensor: shape=(45, 21), dtype=float32, numpy=
array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0

In [170]:
xtrain[0]

'@SammieLynnsMom @tg10781 they will be all done by Sunday trust me *wink*'

In [177]:
lstm_training_model(input_tensor[:1])

<tf.Tensor: shape=(1, 45, 21), dtype=float32, numpy=
array([[[ 1.05008332e-03, -1.77314051e-03, -7.69937818e-04,
         -7.53401371e-04,  8.54153302e-04,  2.78045353e-03,
         -1.46127644e-03, -1.21183484e-03, -2.46877549e-03,
         -2.12072628e-04, -3.64332600e-03,  3.53134144e-03,
          2.04764842e-03,  4.40771040e-03, -4.54714801e-03,
         -4.20295080e-04,  2.46259524e-03, -3.46872094e-03,
          5.95865829e-04, -1.18660147e-03, -2.00384739e-03],
        [ 1.96776865e-03, -1.47862826e-03,  5.77794737e-04,
         -1.00011460e-03,  4.41008524e-05,  3.99911171e-03,
          9.63874569e-04, -1.06681278e-03, -1.70426664e-03,
          5.07433026e-04, -4.78579896e-03,  4.44432208e-03,
          3.08520789e-03,  4.36483044e-03, -5.25411405e-03,
          3.02829169e-04,  4.18807101e-03, -4.53344919e-03,
          2.16157612e-04, -1.83108624e-03, -3.22322780e-03],
        [ 2.73689325e-03, -7.79298280e-05,  1.31027086e-03,
         -1.33363449e-03, -7.25926249e-04,  4

In [171]:
input_tensor = tf.constant(xtrain, dtype=tf.string)

In [180]:
xtrain_numpy = np.array(xtrain)

In [191]:
history = lstm_training_model.fit(xtrain, ytrain_tokenized, epochs=1, batch_size=16)

150/150 [==============================] - 18s 36ms/step - loss: 0.5664


In [192]:
# implement the validation data somehow

In [144]:
entity_ohe([[1,2,1,1]])

<tf.Tensor: shape=(1, 4, 4), dtype=float32, numpy=
array([[[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]]], dtype=float32)>

In [138]:
from tensorflow.keras.layers import TextVectorization

temp_x = [
    ['hello gaurav']
]

temp_layer = TextVectorization(max_tokens=NUM_WORDS, output_sequence_length=40)
temp_layer.adapt(temp_x)


In [102]:
temp_layer(ytrain[:1])

<tf.Tensor: shape=(1, 40), dtype=int64, numpy=
array([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)>

In [105]:
# from tensorflow.keras.layers import CategoryEncoding

# temp_layer_2 = CategoryEncoding(num_tokens=NUM_ENTITIES, output_mode="one_hot")

In [107]:
from tensorflow.keras.layers import Lambda

In [108]:
temp_one_hot_layer = Lambda(lambda x: tf.one_hot(x, 4), output_shape=(None, 4))
temp_one_hot_layer(temp_layer(ytrain[:1]))

<tf.Tensor: shape=(1, 40, 4), dtype=float32, numpy=
array([[[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1.,

In [111]:
tf.one_hot([[1,2,1,0],[0,0,2,1]], 4)

<tf.Tensor: shape=(2, 4, 4), dtype=float32, numpy=
array([[[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]]], dtype=float32)>